*Antonio Coín Castro*

# Métodos Avanzados en Estadística

## Relación 4: clasificación supervisada

### Ejercicio 5

En un grupo de 435 pacientes que habían sufrido quemaduras de tercer grado se midió el área de la zona afectada por las quemaduras (la variable $x$ corresponde a $\log(\text{área} + 1)$). Algunos de los pacientes sobrevivieron ($y=1$) y otros fallecieron ($y=0$). Con el fin de estudiar cómo influye el área de las quemaduras en la probabilidad de supervivencia se ajustó un modelo de regresión logística a los datos con los resultados siguientes:

```
Deviance Residuals:
    Min       1Q   Median       3Q      Max
-2.8518  -0.6998   0.1860   0.5239   2.2089

Coefficients:
             Estimate  Std. Error  z value  Pr(>|z|)
(Intercept)    22.708       2.266   10.020    <2e-16
x             -10.662       1.083   -9.848    <2e-16
---

Null deviance: 524.69 on 433 degrees of freedom
Residual deviance: 335.23 on 432 degrees of freedom
AIC: 339.23
```

**a)** ¿Aportan estos datos evidencia (a nivel $\alpha= 0.01$) de que cuanto mayor es el área afectada menor es la probabilidad de que el individuo sobreviva?

Consideramos el estadístico de Wald para el coeficiente $\hat\beta_1$, 

$$z=\frac{\hat\beta_1}{\widehat {SE}(\hat\beta_1)},$$

donde $\widehat {SE}(\hat\beta_1)$ es un estimador del error típico de $\hat\beta_1$. Sabemos que bajo la hipótesis nula de que $\hat\beta_1=0$ este estadístico sigue una normal estándar, por lo que podemos usar el $p$-valor $p=P(\mathcal N(0,1) > |z|)$ para realizar el contraste de hipótesis al nivel deseado.

En la tabla que proporciona `R` como resumen de los coeficientes del modelo logístico ajustado encontramos los valores estimados de los coeficientes (primera columna), la estimación de sus errores (segunda columna), el valor del estadístico de Wald (tercera columna) y finalmente el $p$-valor asociado (cuarta columna). Atendiendo a la fila para la variable predictiva $x$ (correspondiente a $\beta_1$ en el modelo), vemos que el $p$-valor asociado es $p<2\cdot 10^{-16}$, por lo que al ser este valor mucho más pequeño que $\alpha=0.01$ podemos afirmar que a nivel $0.01$ los datos aportan evidencia de que el coeficiente $\beta_1$ es no nulo, es decir, que la variable $x$ es significativa en el modelo predictivo.

Por otro lado, sabemos que bajo las hipótesis del modelo logístico, el coeficiente $\beta_1$ representa cuánto cambia (en unidades logarítmicas) la razón de probabilidades para una observación genérica $i$ cuando la variable $x_i$ aumenta en una unidad:

$$\log\left(\frac{O_i'}{O_i}\right)=\log\left(\frac{e^{\beta_0+\beta_1(x_i+1)}}{e^{\beta_0+\beta_1x_i}}\right)=\beta_1.$$

En nuestro caso, la estimación para el coeficiente de la variable $x$ es negativa ($\hat\beta_1=-10.662$), por lo que de la expresión anterior se deduce que $O'_i<O_i$, es decir, la razón de probabilidades **disminuye** conforme aumenta la variable $x_i$. Como sabemos que

$$O_i=\frac{\eta_i}{1-\eta_i}=\frac{P(Y_i=1\mid x_i)}{1-P(Y_i=1\mid x_i)}$$

es una función creciente de $\eta_i$, el hecho de que $O_i$ disminuya implica que la propia $\eta_i=P(Y_i=1\mid x_i)$ disminuye cuando $x_i$ aumenta. Recordamos que la variable $x$ medía esencialmente el área afectada por la quemadura (ya que la función $t\mapsto \log(t+1)$ es estrictamente creciente), por lo que concluimos que cuanto mayor es el área afectada, menor es la probabilidad de supervivencia ($Y=1$). Además, esto podemos afirmarlo con nivel de significación $\alpha=0.01$, debido al contraste de hipótesis realizado anteriormente.

**b)** Calcula un intervalo de confianza con nivel aproximado del 95% para el parámetro $\beta_1$, donde $\beta_1$ es el coeficiente correspondiente a la variable $x$.

Por la teoría asintótica de los estimadores de máxima verosimilitud sabemos que para $n$ suficientemente grande se tiene

$$\hat \beta_1 \cong \mathcal N(\beta_1, (\widehat{SE}(\hat \beta_1))^2),$$

luego 

$$\frac{\hat\beta_1 - \beta_1}{\widehat{SE}(\hat \beta_1)} \cong \mathcal N(0, 1).$$

De esta expresión se deduce que los extremos de un intervalo de confianza aproximado al $100(1-\alpha)$% para $\beta_1$ son

$$\hat \beta_1\pm z_{1-\alpha/2}\widehat{SE}(\hat \beta_1),$$

y esta aproximación será mejor conforme mayor sea el número de puntos usado en el ajuste. Utilizando los datos de la salida proporcionada podemos calcular estos extremos para nuestro problema:

In [1]:
beta1_hat <- -10.662
beta1_std <- 1.083
alpha <- 0.05
confidence <- 100*(1-alpha)

len <- qnorm(1-alpha/2)*beta1_std
inf <- beta1_hat - len
sup <- beta1_hat + len
cat("Intervalo de confianza aproximado al", confidence, "%: (", inf, ",", sup, ")")

Intervalo de confianza aproximado al 95 %: ( -12.78464 , -8.539359 )

*Nota:* la estimación concreta $\widehat{SE}(\hat \beta_1)$ corresponde al segundo elemento de la diagonal de la matriz $\hat I=(X'\hat WX)^{-1}$ (matriz de información observada de Fisher), donde $X$ es la matriz de datos extendida (primera columna de $1$s), y $\hat W=\operatorname{diag}\{{\hat \eta_i(1-\hat \eta_i)}\}_{i=1,\dots, n}$, con

$$\hat\eta_i=\hat\eta(x_i)=\frac{1}{1+e^{-\hat\beta_0 - \hat \beta_1x_i}}.$$

**c)** Determina una regla de clasificación para predecir si un individuo sobrevivirá o no en función del valor de la variable $x$. ¿Cuál sería la predicción si $x$ vale $2.5$?

Siguiendo el modelo de regresión logística, predeciremos que un individuo sobrevive cuando $\hat\eta(x)\geq 1-\hat\eta(x)$, es decir, si y solo si $\hat\eta(x)\geq 0.5$. Por tanto, la regla de clasificación será:

$$g(x)=\begin{cases}1, & \hat\eta(x)\geq 0.5,\\
0, & \text{en otro caso}.\end{cases}$$

Podemos simplificar un poco los cálculos recordando que la función logística

$$\sigma(x)=\frac{1}{1+e^{-x}}$$

verifica $\sigma(x)\geq 0.5 \iff x \geq 0$, luego sustituyendo ya nuestros valores de $\hat\beta_0$ y $\hat\beta_1$ tendremos que se predice la clase $Y=1$ cuando $\hat\beta_0 + \hat\beta_1 x =22.708-10.662x \geq 0$, es decir:

$$g(x)=\begin{cases}1, & x \leq 2.13,\\
0, & \text{en otro caso}.\end{cases}$$

Así, predeciremos que un individuo sobrevive cuando el valor $x=\log(\text{área} + 1)$ sea menor o igual que $2.13$, lo cual corresponde a que el área afectada sea menor o igual que $e^{2.13}-1\approx 7.415$ unidades cuadradas. De esta forma, para $x=2.5$ la predicción será que el individuo **no sobrevive**.

### Ejercicio 6

En un experimento descrito en Prentice (1976) se expuso una muestra de escarabajos a cierto pesticida. Tras cinco horas de exposición a distintos niveles de concentración del pesticida algunos de los escarabajos murieron y otros sobrevivieron. Los resultados para cada dosis aparecen en la tabla siguiente:

| Dosis (log CS2 mg/l) | N. insectos | N. muertos |
|:-----------------------------------:|:-----------:|:---------:|
| 1.6907 | 59 | 6  |
| 1.7242 | 60 | 13 |
| 1.7552 | 62 | 18 |
| 1.7842 | 56 | 28 |
| 1.8113 | 63 | 52 |
| 1.8369 | 59 | 53 |
| 1.8610 | 62 | 61 |
| 1.8839 | 60 | 60 |

Formula un modelo de regresión logística para analizar estos datos y estima la probabilidad de que muera un escarabajo expuesto durante cinco horas a una dosis de concentración $1.8$.

Para ajustar un modelo de regresión logística utilizamos la función `glm` (*generalized linear model*), especificando que la variable respuesta se modela como una binomial. Consideramos como variable independiente $X$ la concentración de pesticida a la que fue expuesto el escarabajo durante 5 horas, y como variable respuesta $Y=1$ si el escarabajo muere e $Y=0$ si sobrevive. 

Para introducir los datos tal y como los tenemos necesitamos que la variable `y` esté representada por una matriz con dos columnas, donde la primera representa el número de "éxitos" (muertes en este caso), y la segunda el número de "fracasos".

In [2]:
# Definimos nuestros datos
dosis <- c(1.6907, 1.7242, 1.7552, 1.7842, 
           1.8113, 1.8369, 1.8610, 1.8839)
supervivencia <- data.frame(
    total = c(59, 50, 62, 56, 63, 59, 62, 60),
    muertos = c(6, 13, 18, 28, 52, 53, 61, 60)
)
supervivencia["vivos"] <- supervivencia$total - supervivencia$muertos
y <- as.matrix(supervivencia[c("muertos", "vivos")])

# Definimos el modelo
model <- glm(y~dosis, family=binomial)
summary(model)


Call:
glm(formula = y ~ dosis, family = binomial)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6858  -0.4106   0.8411   1.3552   1.6363  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -59.301      5.166  -11.48   <2e-16 ***
dosis         33.490      2.903   11.54   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 270.712  on 7  degrees of freedom
Residual deviance:  12.944  on 6  degrees of freedom
AIC: 43.085

Number of Fisher Scoring iterations: 4


El modelo logístico ajustado es el siguiente:

$$P(Y=1\mid x)=\frac{1}{1 + e^{-59.301 + 33.490x}}.$$

Para predecir la probabilidad de que un escarabajo muera dado $x=1.8$ utilizamos la función `predict`, especificando `type='response'` para obtener la probabilidad deseada (por defecto muestra el *log-odds*).

In [3]:
prob <- predict(model, data.frame(dosis=1.8), type='response')
prob

1 
0.7274258

Por tanto, la probabilidad de que un escarabajo muera si se expone durante 5 horas a una concentración de pesticida de $1.8 \ [\log CS_2 \ mg/l]$ es de $\approx 0.727$. 

### Ejercicio 9

Sea $Y$ una variable aleatoria tal que $P(Y=0)=P(Y=1)=1/2$, y sea $X$ un vector aleatorio de dimensión $k$ tal que la distribución de $X$ condicionada a $Y=1$ es $\mathcal N_k(0, 2I)$, donde $I$ es la matriz identidad de orden $k$, mientras que la distribución de $X$ condicionada a $Y=0$ es $\mathcal N_k(0,I)$.

**a)** Calcula la regla Bayes para clasificar un vector $x$ como correspondiente a $Y=0$ ó $Y=1$.

Sean $\pi_0=\pi_1=1/2$ las probabilidades a priori para la variable $Y$, y llamemos $f$ a la densidad de $X$, $f_0$ a la densidad de $X\mid Y=0$ y $f_1$ a la densidad de $X\mid Y=1$. La regla de Bayes se define como:

$$g^\ast(x)=\begin{cases}1, & P(Y=1\mid x)\geq P(Y=0\mid x)\\ 0, & \text{en otro caso.}\end{cases}$$

Aplicando la fórmula de Bayes, tenemos que, si $f(x)\neq 0$:

$$P(Y=i\mid x)=\frac{f_i(x)\pi_i}{f(x)}, \quad i=0,1.$$

Usando que las probabilidades a priori coinciden, la regla de Bayes puede reescribirse entonces como:

$$g^\ast(x)=\begin{cases}1, & f_1(x)\geq f_0(x)\\ 0, & \text{en otro caso.}\end{cases}$$

Simplificando, tenemos que:

$$\begin{aligned}
f_1(x)\geq f_0(x) &\iff (2\pi)^{-k/2}2^{-k/2}e^{-\frac{1}{4}x'x} \geq (2\pi)^{-k/2}e^{-\frac{1}{2}x'x}\\
&\iff 2^{-k/2} \geq e^{-\frac{1}{4}\lVert x \rVert^2}\\
&\iff 0 \geq \frac{k}{2}\log 2 - \frac{1}{4}\lVert x \rVert ^2\\
&\iff \lVert x \rVert^2 \geq 2k\log 2.
\end{aligned}$$

Por tanto, la regla de Bayes es:

$$g^\ast(x)=\begin{cases}1, & \lVert x \rVert^2 \geq 2k\log 2\\ 0, & \text{en otro caso.}\end{cases}$$

**b)** Utiliza la regla Bayes para clasificar el punto $x=(2,\dots,2)' \in \mathbb R^k$.

Simplemente debemos comprobar si se verifica la desigualdad $\lVert x \rVert^2 \geq 2k\log 2$, suponiendo $k>0$. Tenemos que:

$$\lVert x \rVert^2 - 2k\log 2 = \sum_{i=1}^k 2^2 - 2k\log 2=4k - 2k\log 2=2k(2-\log 2) > 0,$$
ya que $2>\log 2$. Por tanto, según la regla de Bayes el punto $x$ se clasificaría como $Y=1$.

**c)** Da una expresión del error Bayes (es decir, el error de clasificación de la regla obtenida en el primer apartado) tan explícita como sea posible.

Llamando $c_k := 2k\log 2$, se tiene que:

$$
\begin{aligned}
L^*&=P(g^*(X)\neq Y)\\
&=P(g^*(X)=1, Y=0)+P(g^*(X)=0, Y=1)\\
&= P(g^*(X)=1\mid Y=0)\pi_0 + P(g^*(X)=0\mid Y=1)\pi_1\\
&= \frac{1}{2}\left( P_{X\sim \mathcal N_k(0, I)}\left(\rVert X\lVert^2 \geq c_k\right) + P_{X\sim \mathcal N_k(0, 2I)}\left(\rVert X\lVert^2 < c_k\right) \right).
\end{aligned}
$$

Hacemos ahora las siguientes observaciones:

- Si $X=(X_1,\dots, X_k)'\sim \mathcal N_k(0, I)$, sabemos que $X_1,\dots, X_n$ son normales estándar e independientes. Por otro lado, es conocido que la suma de cuadrados de normales estándar independientes sigue una distribución $\chi^2$. Concretamente:

$$\rVert X \lVert^2 = \sum_{j=1}^k X_j^2 \sim \chi^2_k.$$

- Si $X=(X_1,\dots, X_k)'\sim \mathcal N_k(0, 2I)$, tenemos que $X_1,\dots, X_n$ son independientes y cada una sigue una distribución $\mathcal N(0, 2)$. Si consideramos $Z_1, \dots, Z_k$ normales estándar e independientes, podemos escribir $X_j \equiv \sqrt{2}Z_j$. Por tanto, tenemos las siguientes igualdades en distribución:

$$\rVert X \lVert^2 \equiv \sum_{j=1}^k \left(\sqrt{2}Z_j\right)^2 \equiv 2\sum_{j=1}^k Z_j^2 \equiv 2\chi^2_k.$$

- Finalmente, utilizamos una [propiedad](https://en.wikipedia.org/wiki/Chi-square_distribution#Related_distributions) quizás menos conocida, que asegura que el producto de una constante por una variable con distribución $\chi^2$ sigue una distribución gamma. Concretamente, utilizando la parameterización *shape-scale* de la distribución gamma, tenemos:
$$c\chi^2_k \sim \Gamma(k/2, 2c),$$
por lo que en nuestro caso concreto se verifica que si $X\sim \mathcal N_k(0, 2I)$, entonces

$$\rVert X\lVert^2 \sim \Gamma(k/2, 4).$$

Uniendo todo lo anterior podemos simplificar la expresión obtenida para $L^*$. Si denotamos por $F_Y(\cdot)$ la función de distribución para una variable $Y$, se tiene:

$$\begin{aligned}
L^* &= \frac{1}{2}\left( 1 - F_{\chi^2_k}(c_k) + F_{\Gamma(k/2, 4)}(c_k) \right)\\
&= \frac{1}{2}\left( 1 - \frac{\gamma(k/2, c_k/2)}{\Gamma(k/2)}+ \frac{\gamma(k/2, c_k/4)}{\Gamma(k/2)}\right)\\
&= \frac{1}{2}\left( 1 - \frac{\gamma(k/2, k\log 2) + \gamma(k/2, (k/2) \log 2)}{\Gamma(k/2)} \right),
\end{aligned}$$

donde $\Gamma(s)=\int_0^{\infty}t^{s-1}e^{-t}\, dt$ es la función gamma y $\gamma(s, x)=\int_0^xt^{s-1}e^{-t}\, dt$ es la función gamma incompleta inferior. Finalmente, podemos descomponer casos para hacer alguna simplificación extra, según si $k$ es par o impar.

- Si $k=2n$ con $n>0$, entonces $\Gamma(k/2)=\Gamma(n)=(n-1)!=(k/2 - 1)!$, y por tanto

$$L^* = \frac{1}{2} - \frac{\gamma(k/2, k\log 2) + \gamma(k/2, (k/2) \log 2)}{2(k/2-1)!}.$$

- Si $k=2n+1$ con $n\geq 0$, [se tiene que](https://en.wikipedia.org/wiki/Particular_values_of_the_gamma_function) 
$$\Gamma(k/2)=\Gamma(1/2 + n)=\frac{(2n)!}{4^nn!}\sqrt{\pi} = \frac{(k-1)!}{4^{(k-1)/2}((k-1)/2)!}\sqrt{\pi},$$
y por tanto:

$$L^* =\frac{1}{2} - \frac{\gamma(k/2, k\log 2) + \gamma(k/2, (k/2) \log 2)}{\displaystyle \frac{2(k-1)!}{4^{(k-1)/2}((k-1)/2)!}\sqrt{\pi}}.$$

### Ejercicio 10

Considera un problema de clasificación en el que las dos clases son igualmente probables a priori. La distribución de $X$ condicionada a $Y=1$ es uniforme en el círculo de radio $1$ y centro $(-3, 0)$, mientras que la distribución de $X$ condicionada a $Y=0$ es uniforme en el círculo de radio $1$ y centro $(3,0)$.

**a)** Calcula la regla y el error Bayes correspondientes al problema anterior.

*Nota: consideramos que cuando hablamos de un círculo de centro (a, b) y radio R nos referimos al conjunto* 

$$C((a, b), R)=\{(x, y) \in \mathbb R^2 : (x- a)^2 + (y-b)^2 \leq R^2\}.$$

Siguiendo la misma notación y razonamiento que en el ejercicio anterior, sabemos que la regla de Bayes es:

$$g^*(x)=\begin{cases}1, & \pi_1f_1(x)> \pi_0f_0(x),\\
0, & \text{en otro caso}.\end{cases}$$

Llamamos $C_1=C((-3, 0), 1)$ y $C_0=C((3, 0), 1)$. En esta situación, como las distribuciones condicionadas a la clase son uniformes en un círculo con la misma área, se tiene que:

$$f_0(x)= \frac{1}{\pi}\mathcal I(x \in C_0), \quad f_1(x)=\frac{1}{\pi}\mathcal I(x \in C_1).$$

Entonces, usando que las probabilidades a priori son iguales y que los círculos no tienen intersección, podemos escribir:

$$
\pi_1f_1(x)\geq \pi_0f_0(x) \iff \frac{1}{\pi}\mathcal I(x \in C_1) > \frac{1}{\pi}\mathcal I(x \in C_0)\iff x \in C_1.
$$

Así, la regla de Bayes queda:
$$g^*(x)=\begin{cases}1, & x \in C_1,\\
0, & \text{en otro caso}.\end{cases}$$

El error de Bayes se puede derivar fácilmente:

$$
\begin{aligned}
L^*&=P(g^*(X)\neq Y)\\
&=P(g^*(X)=1\mid Y=0)\pi_0 + P(g^*(X)=0\mid Y=1)\pi_1\\
&= P_{X \sim \mathcal U(C_0)}(x \in C_1)\pi_0 + P_{X \sim \mathcal U(C_1)}(x \notin C_1)\pi_1\\
&= 0 + 0 = 0,
\end{aligned}$$

donde de nuevo hemos usado que los círculos no se intersecan. Esta propiedad de que las clases sean separables ha sido la clave para probar que el error óptimo es $L^*=0$.

**b)** A partir de $n$ datos $D_n \equiv (X_1,Y_1),\dots, (X_n,Y_n)$, se utiliza el clasificador basado en $k=2m+1, m\ge 0$ vecinos más próximos, $g_n$, para predecir la clase $Y$ correspondiente a un punto $X\in \mathbb R^2$. Calcula $P(g_n(X)\neq Y)$. ¿Cuál es el número de vecinos óptimo que hay que considerar en este problema?

Sea $g_n^{(m)}$ el clasificador basado en $k=2m+1$ vecinos más próximos, con $m\geq 0$. En primer lugar, notamos que los círculos que definen las distribuciones condicionadas a la clase están lo suficientemente separados como para que sea fácil expresar el error de cada uno de estos clasificadores. Concretamente, si tenemos un punto $x$ generado por la distribución en $C_i$, cualquier otro punto generado por la misma distribución estará como mucho a distancia $2$ de $x$, y como mínimo a distancia $4$ de cualquier punto generado por la distribución en $C_j$ ($i\neq j$). Por tanto, los $k$ vecinos más cercanos de cualquier punto serán siempre $k$ puntos con la misma etiqueta que él, siempre que existan suficientes de ellos dentro del conjunto de datos $D_n$.

Tomando por ejemplo $m=0$, es evidente que el clasificador $1$-NN cometerá un error al clasificar un punto $X$ cuya clase real es $j$ únicamente cuando no haya ningún punto con $Y_i=j$ en $D_n$. En general, es inmediato comprobar que el clasificador $k$-NN con $k\leq n$ impar cometerá un error cuando haya $\lfloor k/2 \rfloor=\lfloor m+1/2 \rfloor=m$ puntos o menos con etiqueta $j$ en $D_n$, ya que la clasificación se realiza por voto mayoritario entre los vecinos más próximos. Para cubrir también el caso en que $k>n$, consideramos $\tilde m=\min\{m, \lfloor n/2 \rfloor\}$, ya que en este caso no tenemos suficientes vecinos en el conjunto de datos.

Teniendo en cuenta que la variable aleatoria que mide el número de puntos con etiqueta $j$ en $D_n$ puede ser entendida como una binomial $B(n, \pi_j)\equiv B(n, 1/2)$, se tiene:

$$
\begin{aligned}
P(g_n^{(m)}(X)\neq Y)&= \frac{1}{2} \left(P\left(g_n^{(m)}(X)=1\mid Y=0\right) + P\left(g_n^{( m)}=0\mid Y=1\right) \right)\\
&= \frac{1}{2} \left( P\left(\#\{Y_i=0\} \leq \tilde m\right) + P\left(\#\{Y_i=1\} \leq \tilde m\right)\right)\\
&= \frac{1}{2}\left(2\sum_{s=0}^\tilde m\begin{pmatrix} n\\ s\end{pmatrix}\left(\frac{1}{2}\right)^s\left(\frac{1}{2}\right)^{n-s}  \right)\\
&= \frac{1}{2^n}\sum_{s=0}^\tilde m \begin{pmatrix} n\\ s\end{pmatrix}. 
\end{aligned}
$$

Vemos que el error es una función creciente de $m$, luego en este caso la regla óptima (la de menor error esperado) se obtiene tomando $m=0$, es decir, utilizando $k=1$ vecinos más próximos.